# 認識システムの削除

---

認識システムを削除します。学習システムは削除されません。学習システムは、「1.learning_system」の`1920-OpenHPC環境の削除.ipynb`で削除してください。両システムの削除の順番は任意です。

![構成](images/240.destroy_rs.png)

## VCP の初期化

VCPアクセストークンを入力します。

In [ ]:
from getpass import getpass
vcc_access_token = getpass()

VCP SDKの初期化します。エラーになった場合、上のセルを `unfreeze` してから再実行し、正しいアクセストークンを入力してください。

In [ ]:
from common import logsetting
from vcpsdk.vcpsdk import VcpSDK
vcp = VcpSDK(vcc_access_token)

## 削除する認識システムの指定

削除する認識システムの `UnitGroup` 名を指定します。

In [ ]:
rs_ugroup_name = 'TfCpu'

## 環境の削除

### 準備

In [ ]:
# 認識システムのユニットグループ
rs_unit_group = vcp.get_ugroup(rs_ugroup_name)

# 認識システムのパラメータ読み込み
%run scripts/ts/group.py
rs_gvars = load_group_vars(rs_ugroup_name)
print(rs_gvars)

### 作業パラメタの設定

In [ ]:
ssh_vcu_ssh_opts = f'-i {rs_gvars["rs_vcu_prvkey"]} -l root'
ssh_user_ssh_opts = f'-i {rs_gvars["rs_user_prvkey"]}'
rs_user = rs_gvars["rs_user"]

# unit_group.find_ip_addresses() は UnitGroup内の全VCノードのIPアドレスのリストを返します
rs_ipaddress = rs_unit_group.find_ip_addresses(node_state='RUNNING')[0] # １つのVCノードのみ起動したので [0] で最初の要素を取り出す
print(rs_ipaddress)
rs_target = f'{rs_user}@{rs_ipaddress}'

### TensorFlowコンテナの停止

In [ ]:
# TensorFlowコンテナの停止
!ssh {ssh_user_ssh_opts} {rs_target} docker stop tensorflow-{rs_user}

### リソースの削除
ここまで作成した全てのリソース（UnitGroup, Unit、VCノード）を削除します。この操作を行うことで AWS EC2インスタンスやAzure VMなどのクラウドに作成したリソースが削除されます。

> 全てのリソースの削除には 4～5分程度かかります。

In [ ]:
rs_unit_group.cleanup()

削除後の状態の確認。

In [ ]:
# UnitGroupの一覧を DataFrame で表示する
vcp.df_ugroups()

### 認識システムパラメータファイル無効化（リネーム）

In [ ]:
!mv group_vars/{rs_ugroup_name}.yml group_vars/{rs_ugroup_name}.yml.bak